<a href="https://colab.research.google.com/github/cabamarcos/Grupo27_practica2/blob/main/Grupo27_practica2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Importación de librerías y datos

Hacemos los imports correspondientes



In [23]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
with open('attrition_available_27.pkl', 'rb') as archivo:
    datos = pickle.load(archivo)

print(datos)

           hrs  absences  JobInvolvement  PerformanceRating  \
1     7.315971      13.0             2.0                4.0   
6     6.450877      17.0             3.0                4.0   
13    8.871421      14.0             2.0                3.0   
28         NaN       6.0             2.0                3.0   
30    9.662808      11.0             2.0                3.0   
...        ...       ...             ...                ...   
4405  8.316921       6.0             3.0                3.0   
4406  5.897197       NaN             2.0                3.0   
4407       NaN      18.0             3.0                4.0   
4408  9.187612       NaN             2.0                3.0   
4409  6.511790      17.0             4.0                3.0   

      EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance   Age  \
1                         3.0              2.0              4.0  31.0   
6                         1.0              3.0              1.0   NaN   
13                      

#2. EDA

## Visualización

Convertimos los datos en un dataframe

In [3]:
df = pd.DataFrame(datos)

Sacamos el número de instancias y el número de atributos

In [4]:
print("Número de instancias:", len(df))
print("Número de atributos:", len(df.columns))

Número de instancias: 4410
Número de atributos: 31


Vemos el tipo de datos de las instacias

In [5]:
print("Tipos de datos:")
print(df.dtypes)

Tipos de datos:
hrs                        float64
absences                   float64
JobInvolvement             float64
PerformanceRating          float64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
Age                        float64
Attrition                   object
BusinessTravel              object
Department                  object
DistanceFromHome           float64
Education                  float64
EducationField              object
EmployeeCount              float64
EmployeeID                 float64
Gender                      object
JobLevel                   float64
JobRole                     object
MaritalStatus               object
MonthlyIncome              float64
NumCompaniesWorked         float64
Over18                      object
PercentSalaryHike          float64
StandardHours                int64
StockOptionLevel           float64
TotalWorkingYears          float64
TrainingTimesLastYear      float64
Year

Podemos ver que la mayoría de datos son float, pero también observamos objetos y números enteros

## Datos constantes

Queremos ver si tenemos alguna columna con datos constantes

In [6]:
columnas_constantes = []
for columna in df.columns:
    if df[columna].nunique() == 1:
        columnas_constantes.append(columna)

if len(columnas_constantes) > 0:
    print("Las siguientes columnas tienen datos constantes:")
    print(columnas_constantes)
else:
    print("No hay columnas con datos constantes.")

Las siguientes columnas tienen datos constantes:
['EmployeeCount', 'Over18', 'StandardHours']


Podemos ver que tenemos 3 coloumnas con satos constantes:
 - EmployeeCount: Número de empleados en la compañía.
 - Over18: Si los empleados son mayores de 18.
 - StandardHours: El número estándar de horas de trabajo en la empresa.

Esto es bastante logico, ya que "EmployeeCount" indica el numero de empleados en la compañía, y este, en el momento de sacar los datos, no varía. Podría variar si se vuelven a sacar los datos x días después.

"Over18", es normal que tenga valores ctes, ya que es un objeto que puede ser o mayor o menor de 18, y sería muy raro que alguien estuviera trabajando en una empresa con menos de 18 años.

Finalmente "StandardHours" como dice su propio nombre, es el número estandar de horas trabajadas en la compañía. Al ser un número estandar, no es personalizado por lo que siempre es el mismo.

Vamos a ver cual es el dato que se repite en cada una de ellas

In [8]:
print("Primer dato de la columna 'EmployeeCount':", df['EmployeeCount'][0])
print("Primer dato de la columna 'Over18':", df['Over18'][0])
print("Primer dato de la columna 'StandardHours':", df['StandardHours'][0])

Primer dato de la columna 'EmployeeCount': 1.0
Primer dato de la columna 'Over18': Y
Primer dato de la columna 'StandardHours': 8


Vamos a comprobar si esto es cierto

In [9]:
print("Datos de la columna 'EmployeeCount':")
print(df['EmployeeCount'])

print("\nDatos de la columna 'Over18':")
print(df['Over18'])

print("\nDatos de la columna 'StandardHours':")
print(df['StandardHours'])

Datos de la columna 'EmployeeCount':
1       1.0
6       1.0
13      1.0
28      1.0
30      1.0
       ... 
4405    NaN
4406    NaN
4407    1.0
4408    1.0
4409    NaN
Name: EmployeeCount, Length: 4410, dtype: float64

Datos de la columna 'Over18':
1         Y
6         Y
13        Y
28        Y
30      NaN
       ... 
4405      Y
4406      Y
4407      Y
4408      Y
4409      Y
Name: Over18, Length: 4410, dtype: object

Datos de la columna 'StandardHours':
1       8
6       8
13      8
28      8
30      8
       ..
4405    8
4406    8
4407    8
4408    8
4409    8
Name: StandardHours, Length: 4410, dtype: int64


Podemos ver que esto es cierto, pero que hay datos que faltan.

## Datos nulos

Vamos a comprobar el numero total de datos nulos en cada columna.

In [13]:
nulos_por_columna = df.isnull().sum()
porcentaje_nulos_por_columna = (nulos_por_columna / len(df)) * 100

for columna in df.columns:
    print(f"'{columna}': {nulos_por_columna[columna]}, {porcentaje_nulos_por_columna[columna]:.2f}%")

'hrs': 905, 20.52%
'absences': 891, 20.20%
'JobInvolvement': 718, 16.28%
'PerformanceRating': 919, 20.84%
'EnvironmentSatisfaction': 879, 19.93%
'JobSatisfaction': 948, 21.50%
'WorkLifeBalance': 769, 17.44%
'Age': 736, 16.69%
'Attrition': 0, 0.00%
'BusinessTravel': 849, 19.25%
'Department': 782, 17.73%
'DistanceFromHome': 747, 16.94%
'Education': 715, 16.21%
'EducationField': 710, 16.10%
'EmployeeCount': 822, 18.64%
'EmployeeID': 796, 18.05%
'Gender': 932, 21.13%
'JobLevel': 782, 17.73%
'JobRole': 782, 17.73%
'MaritalStatus': 0, 0.00%
'MonthlyIncome': 929, 21.07%
'NumCompaniesWorked': 936, 21.22%
'Over18': 924, 20.95%
'PercentSalaryHike': 788, 17.87%
'StandardHours': 0, 0.00%
'StockOptionLevel': 708, 16.05%
'TotalWorkingYears': 898, 20.36%
'TrainingTimesLastYear': 887, 20.11%
'YearsAtCompany': 956, 21.68%
'YearsSinceLastPromotion': 0, 0.00%
'YearsWithCurrManager': 895, 20.29%


Se puede observar que en ciertas columnas no hay ni un solo missing value, mientras que en otras, el % de missing values puede llegar hasta el 0,21%, más de un quinto de los datos faltan.

Esto nos lleva a tener que pensar una estrategia para ver que podemos hacer con esos datos que faltan

Primero vamos a realizar un análisis estadístico

In [20]:
# Calcula las estadísticas descriptivas para cada variable
desc_stats = df.describe()
print(desc_stats)

               hrs     absences  JobInvolvement  PerformanceRating  \
count  3505.000000  3519.000000     3692.000000        3491.000000   
mean      7.318161    12.709292        2.729415           3.153538   
std       1.330881     5.492969        0.710773           0.360557   
min       5.416880     1.000000        1.000000           3.000000   
25%       6.281167     8.000000        2.000000           3.000000   
50%       7.027552    13.000000        3.000000           3.000000   
75%       7.923872    17.000000        3.000000           3.000000   
max      10.937261    24.000000        4.000000           4.000000   

       EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance          Age  \
count              3531.000000      3462.000000      3641.000000  3674.000000   
mean                  2.719060         2.710861         2.751991    36.950191   
std                   1.093544         1.111206         0.712342     9.135962   
min                   1.000000         1.0000